In [13]:
from ultralytics import YOLO
import cv2
# Загрузка предобученной модели YOLOv8
model = YOLO('yolov8n.pt')  # 'n' — nano версия (самая легкая)

In [14]:
image = cv2.imread('ships_3.jpg')

results = model(image)

# Инициализация счетчика кораблей
tv_count = 0

# Перебираем все обнаруженные объекты
for result in results:
    for box in result.boxes:
        class_id = int(box.cls)
        if class_id == 8:  # Нужный класс в COCO
            tv_count += 1

# Визуализация результатов с bounding boxes
annotated_image = results[0].plot()

# Добавляем текст с количеством кораблей
text = f'SHIPS Count: {tv_count}'
cv2.putText(annotated_image, text, (850, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (180, 20, 220), 2)

#cv2.imwrite('./data/tv_data/detected_tvs/detected_tvs.jpg', annotated_image)

# Показываем результат 
cv2.imshow('TV Detection', annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Выводим количество
#print(f"Найдено кораблей: {tv_count}")


0: 448x640 3 boats, 127.0ms
Speed: 6.9ms preprocess, 127.0ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)


In [17]:
%tb
from flask import Flask, request, jsonify, send_from_directory
from ultralytics import YOLO
import cv2
import tempfile
import os
from flask_cors import CORS

import shutil

# Путь к папке временных файлов
TEMP_DIR = "temporary_folder"

# Очистка папки перед запуском (опционально)
shutil.rmtree(TEMP_DIR, ignore_errors=True)
os.makedirs(TEMP_DIR, exist_ok=True)

app = Flask(__name__)
CORS(app)  # Разрешить запросы с любых доменов
model = YOLO('yolov8n.pt')

@app.route('/')
def home():
    return "TV Detection API is running! Use /process endpoint to upload video."

@app.route('/process', methods=['POST'])
def process_video():
    try:
        if 'video' not in request.files:
            return jsonify({'error': 'No video uploaded'}), 400

        video_file = request.files['video']
        if video_file.filename == '':
            return jsonify({'error': 'Empty filename'}), 400

        # Сохраняем входное видео
        input_path = os.path.join(TEMP_DIR, "input_video.mp4")
        video_file.save(input_path)

        # Инициализация видео-потоков
        cap = cv2.VideoCapture(input_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Выходной файл
        output_filename = "output_video.mp4"
        output_path = os.path.join(TEMP_DIR, output_filename)

        fourcc = cv2.VideoWriter_fourcc(*'avc1')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        counts = []

        # Обработка кадров
        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                results = model(frame, classes=[8])
                count = len(results[0].boxes)
                counts.append(count)

                annotated_frame = results[0].plot()
                out.write(annotated_frame)
        finally:
            # Гарантированное освобождение ресурсов
            cap.release()
            out.release()

        # Удаление временных файлов (опционально)
        #os.remove(input_path)
        #os.remove(output_path)

        return jsonify({
            'video_url': f'/download/{output_filename}',
            'counts': counts
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/download/<filename>')
def download_file(filename):
    return send_from_directory(
        TEMP_DIR,
        filename,
        mimetype='video/mp4',  # Явно указываем MIME-тип
        as_attachment=False
    )

if __name__ == '__main__':
    app.run()

NameError: name 'annotated_frame' is not defined

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Apr/2025 23:37:34] "GET /download/output_video.mp4 HTTP/1.1" 404 -
127.0.0.1 - - [30/Apr/2025 23:40:55] "OPTIONS /process HTTP/1.1" 200 -



0: 384x640 2 boats, 174.9ms
Speed: 7.0ms preprocess, 174.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 125.1ms
Speed: 7.0ms preprocess, 125.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 109.0ms
Speed: 2.0ms preprocess, 109.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 111.8ms
Speed: 3.2ms preprocess, 111.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 105.0ms
Speed: 2.0ms preprocess, 105.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 111.9ms
Speed: 3.0ms preprocess, 111.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 104.1ms
Speed: 3.0ms preprocess, 104.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 117.9ms
Speed: 5.0ms preprocess, 117.9ms inference, 1.0ms postprocess per image at shape (

127.0.0.1 - - [30/Apr/2025 23:41:13] "POST /process HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2025 23:41:13] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:41:13] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:41:13] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:41:14] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:41:14] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:41:14] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:41:14] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:41:35] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:42:41] "OPTIONS /process HTTP/1.1" 200 -



0: 384x640 2 boats, 150.0ms
Speed: 6.8ms preprocess, 150.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 139.0ms
Speed: 3.0ms preprocess, 139.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 104.0ms
Speed: 3.0ms preprocess, 104.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 106.5ms
Speed: 4.0ms preprocess, 106.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 130.0ms
Speed: 2.0ms preprocess, 130.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 124.4ms
Speed: 2.7ms preprocess, 124.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 124.0ms
Speed: 3.0ms preprocess, 124.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 126.0ms
Speed: 5.0ms preprocess, 126.0ms inference, 1.0ms postprocess per image at shape (

127.0.0.1 - - [30/Apr/2025 23:42:58] "POST /process HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2025 23:42:58] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:42:59] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:44:36] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:44:36] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:44:37] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:44:37] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:44:37] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:44:37] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:44:37] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:44:37] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:44:37] "GET /download/output_video.mp4 HTTP/1.1" 206


0: 384x640 2 boats, 201.5ms
Speed: 5.0ms preprocess, 201.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 180.0ms
Speed: 25.0ms preprocess, 180.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 122.0ms
Speed: 4.0ms preprocess, 122.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 117.7ms
Speed: 2.0ms preprocess, 117.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 107.0ms
Speed: 5.0ms preprocess, 107.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 103.0ms
Speed: 1.9ms preprocess, 103.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 boats, 111.0ms
Speed: 4.0ms preprocess, 111.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 boats, 104.0ms
Speed: 3.0ms preprocess, 104.0ms inference, 1.0ms postprocess per image at shape 

127.0.0.1 - - [30/Apr/2025 23:49:49] "POST /process HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2025 23:49:49] "GET /download/output_video.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2025 23:49:50] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:49:50] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:49:50] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:49:50] "GET /download/output_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [30/Apr/2025 23:49:50] "GET /download/output_video.mp4 HTTP/1.1" 206 -


In [4]:
!pip install flask_cors

  Obtaining dependency information for flask_cors from https://files.pythonhosted.org/packages/85/61/4aea5fb55be1b6f95e604627dc6c50c47d693e39cab2ac086ee0155a0abd/flask_cors-5.0.1-py3-none-any.whl.metadata
